# Greetings!

My name is Devdhar Patel and I am a graduate student at University of Massachusetts Amherst. This is an assignment for  COMPSCI 690V Visual analytics.

For this assignment, I will be using a dataset on gapminder. (https://docs.google.com/spreadsheets/d/15Fkl9TO8SG_5ap8AFB-tT4TsOLnCZAoGXYXYtH9psbU/pub)


#### Importing the dataset:

In [12]:
import pandas
from bokeh.layouts import column
from bokeh.plotting import Figure, output_notebook, show 
import math
from bokeh.palettes import Category10
from bokeh.models import HoverTool,CustomJS, Slider, ColumnDataSource
from bokeh.layouts import widgetbox
from bokeh.io import push_notebook

dataframe = pandas.read_excel(io='Homicide age adjusted indicator LIVE -05 20100919.xlsx',sheetname='Data',
                              index_col=0,na_values='NA')
dataframe.columns = dataframe.columns.astype(str)
countries = list(dataframe.index)
years = list(dataframe.columns.values)

#### For my first graph, I will be creating a scatter plot for the murder rate for all countries in 2005 : 


In [15]:
source = ColumnDataSource(data=dict(
    x=countries,
    y=dataframe['2005'],
    desc=countries
))
dataFrameSource = ColumnDataSource(dataframe);
callback = CustomJS(args=dict(source=source,dataFrameSource=dataFrameSource), code="""
    var data = source.data;
    var y = data['y'];
    for(var i = 0; i< y.length; i++ ){
        y[i] = dataFrameSource.data[cb_obj.value.toString()][i];
    }
    source.change.emit();
""")
    

slider = Slider(start=1950, end=2005, value=2005, step=1, title="Year")
slider.js_on_change('value', callback)


hover = HoverTool(tooltips="@desc")

p = Figure(x_range = countries, x_axis_label='Country', y_axis_label='Murder per 100,000, age adjusted', 
           plot_width=3000, title='Murder rate for countries in the year '+str(slider.value),tools=[hover])
p.xaxis.major_label_orientation = math.pi/2

p.circle('x','y', size=10, color="navy", alpha=0.5, source=source)



output_notebook()    
layout = column(slider, p)
show(layout)


Loading BokehJS ...

In [16]:
null_values = dataframe.isnull().sum(axis=1);
null_values = pandas.DataFrame({'Null values':null_values.values},index=null_values.index);
null_values.index.name = None;
null_values = null_values.sort_values(by='Null values',ascending=True)

print("rows with the least null values:")

null_values.head()
focus_countries = null_values.head();
focus_countries


rows with the least null values:


,Null values
Ireland,0
United States,0
New Zealand,0
Netherlands,0
United Kingdom,0


#### The second graph is a line graph of murder rates in United States from 1950 to 2005:

In [18]:
hover2 = HoverTool(tooltips="@desc");
line = Figure(x_range = years, x_axis_label='Year', y_axis_label='Murder per 100,000, age adjusted', 
              title="Murder rates for USA across the years", plot_width=1500,tools=[hover2])
source2 = ColumnDataSource(data=dict(
            x=years,
            y=list(dataframe.loc["United States", : ]),
            desc=list(dataframe.loc["United States", : ]),
        ))
line.line(x='x',y='y',source=source2,line_width=2)
show(line)

#### The following graph shows a multiline graph for multiple countries:

In [21]:
focus_countries_names = list(focus_countries.index);
hover2 = HoverTool(tooltips="@desc");
multiline = Figure(x_range = years, x_axis_label='Year', y_axis_label='Murder per 100,000, age adjusted',
                   title="Murder rates for countries across the years", plot_width=1500,tools=[hover2])
for index in range(0,len(focus_countries_names)): 
    tempSource = ColumnDataSource(data=dict(
            x=years,
            y=list(dataframe.loc[focus_countries_names[index], : ]),
            desc=list(dataframe.loc[focus_countries_names[index], : ]),
        ))
    multiline.line('x','y',line_width=2, color=Category10[5][index],
                      legend=focus_countries_names[index], source=tempSource)

multiline.legend.location = "top_left"
multiline.legend.click_policy="hide"
show(multiline)